# GANS Project

In [1]:
# install if needed
#!pip install sqlalchemy
#!pip install pymysql

#pip install lat-lon-parser

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lat_lon_parser import parse    # for decimal coordinates
from datetime import datetime, timedelta
import pytz
import re
import password_collection

In [3]:
#Extract the table of city from SQL database
def connection():
    schema = "Gans_workshop"
    host = password_collection.google_cloud_ip
    user = "root"
    password = password_collection.google_cloud_password
    port = 3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    return connection_string

### Static Database

Function to create a **City** dataframe

In [4]:
def removed_duplicate(table_name,
                 data,
                 composite_key,
                 connection_string):
    
    new_data = data.copy()
    # Pull old data from database
    old_data = pd.read_sql(table_name,
                           con=connection_string)
    
    # Create composite key column for new data and old data
    # (Data that matches values in these columns will not be appended)
    old_data['composite_key'] = old_data.apply(lambda row: ''.join(str(row[column]) for column in composite_key), axis=1)
    new_data['composite_key'] = new_data.apply(lambda row: ''.join(str(row[column]) for column in composite_key), axis=1)
    
    # Subtract set of composite keys of old data from set of new data
    old_keys = set(old_data['composite_key'])
    new_keys = set(new_data['composite_key'])
    fresh_data_keys = new_keys - old_keys
    
    # Use .loc to select and then append the fresh data
    fresh_data = new_data.loc[new_data['composite_key'].isin(fresh_data_keys)].copy()
    fresh_data.drop(columns='composite_key', inplace=True)
    fresh_data.to_sql(table_name,
                      con=connection_string,
                      if_exists='append',
                      index=False)
    

In [5]:
def city_dataframe(cities):

    connection_string = connection()
    city_data = []

    for city in cities:
        url = f"https://www.wikipedia.org/wiki/{city}"
        response = requests.get(url)
        city_soup = BeautifulSoup(response.content, 'html.parser')
    
        # extract the relevant information
        city_latitude = city_soup.find(class_="latitude").get_text()
        city_longitude = city_soup.find(class_="longitude").get_text()
        country = city_soup.find(class_="infobox-data").get_text(strip=True) #Removing the unicode from our results (if they exist)
    
        # keep track of data per city
        city_data.append({"city_name": city,
                        "country": country,
                        "latitude": parse(city_latitude), # latitude in decimal format
                        "longitude": parse(city_longitude), # longitude in decimal format
                        })
    city_data_df = pd.DataFrame(city_data)

    #Create the SQL database for City Data
    composite_key = ['city_name', 'country']
    removed_duplicate('city',
                 city_data_df,
                 composite_key,
                 connection_string)
    city_sql = pd.read_sql('city', con = connection_string)
      
    return city_sql

Function to create **Airport Codes** connecting to the cities

In [6]:
def airport_code_dataframe():

    connection_string = connection()
    airport_code = pd.DataFrame()
    
    #Extract the table of city from SQL database
    city_sql = pd.read_sql('city', con = connection_string)    
    
    for i, row in city_sql.iterrows():
        city_id = row['city_id']
        lat = row['latitude']
        lon = row['longitude']
        
        airport_code_url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat}/{lon}/km/50/16"
        airport_code_string = {"withFlightInfoOnly":"true"}
        headers = {
            "X-RapidAPI-Key": password_collection.aerodata_key,
            "X-RapidAPI-Host": password_collection.aerodata_host
        }
        airport_code_api = requests.request("GET", airport_code_url, headers=headers, params=airport_code_string)
    
        airport = pd.json_normalize(airport_code_api.json()['items'])
        airport['city_id'] = city_id
        airport_code = pd.concat([airport_code, airport], ignore_index=True)
    
    airport_code_df = airport_code.drop(columns = {'shortName'}).rename(columns = {'timeZone': 'timezone', 'name': 'airport_name', 'municipalityName': 'airport_city', 'countryCode': 'airport_country', 'location.lat': 'airport_lat', 'location.lon': 'airport_lon'})
    airport_city_df = airport_code_df[['icao', 'iata', 'city_id']]
    airport_code_df = airport_code_df.drop(columns = {'city_id'})
    airport_code_df = airport_code_df.drop_duplicates()
    
    ## Create SQL table for Airport Codes    
    composite_key1 = ['icao']
    removed_duplicate('airport_code',
                 airport_code_df,
                 composite_key1,
                 connection_string)
    composite_key2 = ['city_id', 'icao']
    removed_duplicate('airport_city',
                    airport_city_df,
                     composite_key2,
                     connection_string)
    
    
    airport_code_sql = pd.read_sql('airport_code', con = connection_string)
    airport_city_sql = pd.read_sql('airport_city', con = connection_string)
    
    return airport_code_sql, airport_city_sql

In [7]:
def update_static(cities):

    city_sql = city_dataframe(cities)
    airport_code_sql, airport_city_sql = airport_code_dataframe()

    return city_sql, airport_code_sql, airport_city_sql

### Dynamic Database

Function to create a **Population** database

In [8]:
def population_dataframe():
    
    connection_string = connection()
    population_data = []
    retrieved = datetime.today().strftime('%Y-%m-%d')
    
    #Extract the table of city from SQL database
    city_sql = pd.read_sql('city', con = connection_string)

    #Collecting the Lat & Lon of the cities to fetch weather forecast info
    for i, row in city_sql.iterrows():
        city_id = row['city_id']
        city = row['city_name']
    
        url = f"https://www.wikipedia.org/wiki/{city}"
        response = requests.get(url)
        city_soup = BeautifulSoup(response.content, 'html.parser')
    
        # extract the relevant information
        city_population = city_soup.find(string="Population").find_next("td").get_text()
        city_population_clean = city_population.replace(",", "")
    
        # for each city we append a dictionary of values to the list
        population_data.append({"population": int(city_population_clean),
                                "retrieved": retrieved,
                                "city_id": city_id
                               })
    population_data_df = pd.DataFrame(population_data)
    
    #Create the SQL database for Population per city Data
    population_data_df.to_sql('population',
                  if_exists='append',
                  con=connection_string,
                  index=False)
    population_sql = pd.read_sql('population', con = connection_string)
      
    return population_sql

A Function for a **Weather Forcast** database

In [9]:
def weather_forecast_dataframe():

    connection_string = connection()
    city_forecast = []
    retrieved = datetime.today().strftime('%Y-%m-%d')
    
    #Extract the table of city from SQL database
    city_sql = pd.read_sql('city', con = connection_string)

    #Collecting the Lat & Lon of the cities to fetch weather forecast info
    for i, row in city_sql.iterrows():
        city_id = row['city_id']
        city = row['city_name']
        lat = row['latitude']
        lon = row['longitude']

        #APIs to collect weather forecast info requiring lat & lon input 
        weather_url = "https://api.openweathermap.org/data/2.5/forecast"
        weather_string = {"lat": lat, 
                   "lon": lon, 
                   'appid': password_collection.weather_key,
                   'units': 'metric'}
        weather_api = requests.request('GET',
                                       weather_url,
                                       params = weather_string)
        
        weather_json = weather_api.json()
          
        #Selecting only specific values to built a database on SQL for weather forecast
        for forecast in weather_json['list']:
            city_weather = {
                'city_id': city_id,
                'forecast_time': forecast['dt_txt'],
                'temperature': forecast['main'].get('temp'),
                'forecast_id': forecast['weather'][0].get('id'),
                'forecast': forecast['weather'][0].get('main'),
                'forecast_desc': forecast['weather'][0].get('description'),
                'humidity': forecast['main'].get('humidity'),
                'precipitation_percent':  forecast.get('pop', 0),
                'cloud_percent': forecast.get('clouds', {}).get('all', 0),
                'rain_in_last_3hr_mm': forecast.get('rain', {}).get('3h', 0),
                'snow_in_last_3hr_mm': forecast.get('snow', {}).get('3h', 0),
                'wind_speed': forecast.get('wind', {}).get('speed', 0),
                'visibility': forecast.get('visibility', 0),
                'retrieved': retrieved
            }
            
            city_forecast.append(city_weather)
    city_forecast_df = pd.DataFrame(city_forecast)

    #Creating SQL database for weather forecast  
    city_forecast_df.to_sql('weather_forecast',
                      if_exists='append',
                      con=connection_string,
                      index=False)
    weather_forecast_sql = pd.read_sql('weather_forecast', con = connection_string)
    
    return weather_forecast_sql

Function to create an **Arriving Flight** database

In [3]:
def arrival_flight_dataframe():
    
    connection_string = connection()
    arrival_flight = []
    retrieved = datetime.today().strftime('%Y-%m-%d')

    #Extract the table of city from SQL database
    airport_code_sql = pd.read_sql('airport_code', con = connection_string)
    
    #collegting 'icao' and 'timezone' data
    for i, row in airport_code_sql.iterrows():
        icao = row['icao']
        timezone = row['timezone']
    
        #creating the datetime periods for tomorrow from the local time (12-hour daytime + 12-hour nighttime)
        tomorrow = datetime.now(pytz.timezone(timezone)) + timedelta(days=1)
        time_input = [[tomorrow.strftime('%Y-%m-%dT00:00'), tomorrow.strftime('%Y-%m-%dT11:59')], [tomorrow.strftime('%Y-%m-%dT12:00'), tomorrow.strftime('%Y-%m-%dT23:59')]]
    
        for start, end in time_input:
            flight_url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{start}/{end}"
            flight_string = {"withLeg":"false",
                           "direction":"Arrival",
                           "withCancelled":"false",
                           "withCodeshared":"false",
                           "withCargo":"false",
                           "withPrivate":"false",
                           "withLocation":"false"}
            headers = {
            	"X-RapidAPI-Key": password_collection.aerodata_key,
                "X-RapidAPI-Host": password_collection.aerodata_host
            }
            
            flight_api = requests.get(flight_url, headers=headers, params=flight_string)
            flight_json = flight_api.json()
    
            for flight in flight_json['arrivals']:
                flight_list = {
                    'flight_no': flight.get('number', 'unidentified'),
                    'airline': flight.get('airline', 'unidentified').get('name', 'unidentified'),
                    'arrival_time': flight['movement'].get('revisedTime', flight['movement']['scheduledTime'])['local'].split('+')[0],
                    'airport_from': flight['movement']['airport'].get('icao', 'unidentified'),
                    'airport_to': icao,
                    'retrieved': retrieved
                }
                arrival_flight.append(flight_list)
            
    arrival_flight_df = pd.DataFrame(arrival_flight)

    #Creating SQL database for arrival flights    
    arrival_flight_df.to_sql('arrival_flight',
                  if_exists='append',
                  con=connection_string,
                  index=False)
    arrival_flight_sql = pd.read_sql('arrival_flight', con = connection_string)

    return arrival_flight_sql

# Update functions

In [11]:
city_list = ["Berlin", "Hamburg", "Munich", "Bangkok"]

In [12]:
city_sql, airport_code_sql, airport_city_sql = update_static(city_list)

In [13]:
city_sql

,city_id,city_name,country,latitude,longitude
0,1,Berlin,Germany,52.5200,13.405
1,2,Hamburg,Germany,53.5500,10.000
2,3,Munich,Germany,48.1375,11.575
3,4,Bangkok,Thailand,13.7525,100.494


In [14]:
airport_code_sql

,icao,iata,airport_name,airport_city,airport_country,timezone,airport_lat,airport_lon
0,EDDB,BER,Berlin Brandenburg,Berlin,DE,Europe/Berlin,52.3514,13.49390
1,EDDH,HAM,Hamburg,Hamburg,DE,Europe/Berlin,53.6304,9.98823
2,EDDM,MUC,Munich,Munich,DE,Europe/Berlin,48.3538,11.78610
3,VTBD,DMK,Bangkok Don Mueang,Bangkok,TH,Asia/Bangkok,13.9126,100.60700
4,VTBS,BKK,Bangkok Suvarnabhumi,Bangkok,TH,Asia/Bangkok,13.6811,100.74700


In [15]:
airport_city_sql

,airport_city_id,city_id,icao,iata
0,1,1,EDDB,BER
1,2,2,EDDH,HAM
2,3,3,EDDM,MUC
3,4,4,VTBD,DMK
4,5,4,VTBS,BKK


------------------------------------------------------------------------------
# Example of Cloud Function

import functions_framework
import pandas as pd
import requests
from datetime import datetime, timedelta
import re
import password_collection

@functions_framework.http
def weather_forecast(request):

    connection_string = connection()
    city_forecast = []
    retrieved = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    #Extract the table of city from SQL database
    city_sql = pd.read_sql('city', con = connection_string)

    #Collecting the Lat & Lon of the cities to fetch weather forecast info
    for i, row in city_sql.iterrows():
        city_id = row['city_id']
        city = row['city_name']
        lat = row['latitude']
        lon = row['longitude']

        #APIs to collect weather forecast info requiring lat & lon input 
        weather_url = "https://api.openweathermap.org/data/2.5/forecast"
        weather_string = {"lat": lat, 
                   "lon": lon, 
                   'appid': password_collection.weather_key,
                   'units': 'metric'}
        weather_api = requests.request('GET',
                                       weather_url,
                                       params = weather_string)
        
        weather_json = weather_api.json()
          
        #Selecting only specific values to built a database on SQL for weather forecast
        for forecast in weather_json['list']:
            city_weather = {
                'city_id': city_id,
                'forecast_time': forecast['dt_txt'],
                'temperature': forecast['main'].get('temp'),
                'forecast_id': forecast['weather'][0].get('id'),
                'forecast': forecast['weather'][0].get('main'),
                'forecast_desc': forecast['weather'][0].get('description'),
                'humidity': forecast['main'].get('humidity'),
                'precipitation_percent':  forecast.get('pop', 0),
                'cloud_percent': forecast.get('clouds', {}).get('all', 0),
                'rain_in_last_3hr_mm': forecast.get('rain', {}).get('3h', 0),
                'snow_in_last_3hr_mm': forecast.get('snow', {}).get('3h', 0),
                'wind_speed': forecast.get('wind', {}).get('speed', 0),
                'visibility': forecast.get('visibility', 0),
                'retrieved': retrieved
            }
            
            city_forecast.append(city_weather)
    city_forecast_df = pd.DataFrame(city_forecast)

    #Creating SQL database for weather forecast  
    city_forecast_df.to_sql('weather_forecast',
                      if_exists='append',
                      con=connection_string,
                      index=False)
    
    return 'Weather Forecast Updated Successfully'

#Extract the table of city from SQL database
def connection():
    schema = "Gans_workshop"
    host = password_collection.google_cloud_ip
    user = "root"
    password = password_collection.google_cloud_password
    port = 3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    return connection_string

### Requirement file

functions-framework==3.*
functions-framework
SQLAlchemy
PyMySQL
pandas
requests